In [1]:
import pandas as pd

Analyse node categories and prefixes from lcc merge file

In [ ]:
nodes = pd.read_csv("../../data/processed/finals/lcc_nodes.tsv", sep="\t", low_memory=False)

nodes["source"] = nodes["source"].str.split("|")

nodes = nodes.explode(["source"])
nodes["prefixes"] = nodes.id.str.split(":").str[0]

categories = nodes[["source", "category"]].groupby(["category","source"]).size().unstack(fill_value=0)

prefixes = nodes[["source", "prefixes"]].groupby(["prefixes","source"]).size().unstack(fill_value=0)

display(categories)
display(prefixes)

Analyse edges predicates from lcc merge file

In [ ]:
edges = pd.read_csv("../../data/processed/finals/lcc_edges.tsv", sep="\t", low_memory=False)

edges["source"] = edges["source"].str.split("|")

edges = edges.explode(["source"])

predicates = edges[["source", "predicate"]].groupby(["predicate","source"]).size().unstack(fill_value=0)

predicates.to_csv("lcc_predicates.tsv", sep="\t")
display(predicates)

Create pivot table for Sankey Graph from lcc file

In [ ]:
edges = pd.read_csv("../../data/processed/finals/lcc_edges.tsv", sep="\t", low_memory=False)[["subject", "object"]]
edges["subject"] = edges["subject"].str.split(":").str[0]
edges["object"] = edges["object"].str.split(":").str[0]
final = edges.pivot_table(columns=['subject','object'], aggfunc='size')
display(final)